In [1]:
import numpy as np
import pandas as pd
import spacy
import string
import pickle as pkl
from sklearn import preprocessing
import os.path

In [2]:
#load in the data
train_df = pd.read_csv('snli_train.tsv', sep="\t")
val_df = pd.read_csv('snli_val.tsv',sep="\t")

#get data & convert sentences to lists
train_sentence1=train_df['sentence1'].values.tolist()
train_sentence2=train_df['sentence2'].values.tolist()
val_sentence1=val_df['sentence1'].values.tolist()
val_sentence2=val_df['sentence2'].values.tolist()

#convert the text labels to numeric
le = preprocessing.LabelEncoder()
le.fit(train_df['label'])
train_targets=le.transform(train_df['label']).tolist()
val_targets=le.transform(val_df['label']).tolist()

In [3]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
    tokens = tokenizer(sent)
    return [token.text.lower() for token in tokens if (token.text not in punctuations)]

In [4]:
# This is the code cell that tokenizes train/val/test datasets
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

In [7]:
#train set tokens
if not os.path.exists('train_all_combined_sentence_tokens.p'):
    print("Tokenizing train data")
    train_sentence1_tokens, train_all_sentence1_tokens = tokenize_dataset(train_sentence1)
    train_sentence2_tokens, train_all_sentence2_tokens = tokenize_dataset(train_sentence2)
    train_all_combined_sentence_tokens = train_all_sentence1_tokens + train_all_sentence2_tokens
    pkl.dump(train_all_combined_sentence_tokens, open("train_all_combined_sentence_tokens.p", "wb"))

In [8]:
#if pickle files do not exist, collect them.  Otherwise, this if block will not run
if not os.path.exists('train_sentence1_tokens.p'):
    print ("Tokenizing train data")
    train_sentence1_tokens, train_all_sentence1_tokens = tokenize_dataset(train_sentence1)
    train_sentence2_tokens, train_all_sentence2_tokens = tokenize_dataset(train_sentence2)
    pkl.dump(train_sentence1_tokens, open("train_sentence1_tokens.p", "wb"))
    pkl.dump(train_sentence2_tokens, open("train_sentence2_tokens.p", "wb"))
    #pkl.dump(train_all_sentence1_tokens, open("train_all_sentence1_tokens.p", "wb"))
    #pkl.dump(train_all_sentence2_tokens, open("train_all_sentence2_tokens.p", "wb"))1

    #combine tokens from both sentences to create a shared dictionary
    train_all_combined_sentence_tokens = train_all_sentence1_tokens + train_all_sentence2_tokens
    pkl.dump(train_all_combined_sentence_tokens, open("train_all_combined_sentence_tokens.p", "wb"))

    #val set tokens
    print ("Tokenizing val data")
    val_sentence1_tokens, _ = tokenize_dataset(val_sentence1)
    val_sentence2_tokens, _ = tokenize_dataset(val_sentence2)
    pkl.dump(val_sentence1_tokens, open("val_sentence1_tokens.p", "wb"))
    pkl.dump(val_sentence2_tokens, open("val_sentence2_tokens.p", "wb"))

In [9]:
#If you have previously run the previous cell, run this cell instead to load preprocessed datasets
train_sentence1_tokens = pkl.load(open("train_sentence1_tokens.p", "rb"))
train_sentence2_tokens = pkl.load(open("train_sentence2_tokens.p", "rb"))
train_all_combined_sentence_tokens = pkl.load(open("train_all_combined_sentence_tokens.p", "rb"))
train_all_sentence1_tokens = pkl.load(open("train_all_sentence1_tokens.p", "rb"))
train_all_sentence2_tokens = pkl.load(open("train_all_sentence2_tokens.p", "rb"))
val_sentence1_tokens = pkl.load(open("val_sentence1_tokens.p", "rb"))
val_sentence2_tokens = pkl.load(open("val_sentence2_tokens.p", "rb"))

In [10]:
#print information about the token datasets
# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_tokens)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_tokens)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_tokens)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_tokens)))

print ("\nTotal number of tokens in sentence1 train dataset is {}".format(len(train_all_sentence1_tokens)))
print ("Total number of tokens in sentence2 train dataset is {}".format(len(train_all_sentence2_tokens)))
print ("Total number of tokens in combined sent1 & sent2 train dataset is {}".format(len(train_all_combined_sentence_tokens)))

print ("\nTotal number of *unique* tokens in sentence1 train dataset is {}".format(len(set(train_all_sentence1_tokens))))
print ("Total number of *unique* tokens in sentence2 train dataset is {}".format(len(set(train_all_sentence2_tokens))))
print ("Total number of *unique* tokens in sent1 & sent2 train dataset is {}".format(len(set(train_all_combined_sentence_tokens))))


Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000

Total number of tokens in sentence1 train dataset is 1294135
Total number of tokens in sentence2 train dataset is 743372
Total number of tokens in combined sent1 & sent2 train dataset is 2037507

Total number of *unique* tokens in sentence1 train dataset is 14131
Total number of *unique* tokens in sentence2 train dataset is 15225
Total number of *unique* tokens in sent1 & sent2 train dataset is 19642


In [11]:
#build vocabularies for sentence1 and sentence2
from collections import Counter

max_vocab_size = 10000
# save index 0 for unk and 1 for pad
PAD_IDX = 0
UNK_IDX = 1

def build_vocab(all_tokens):
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    token_counter = Counter(all_tokens)
    vocab, count = zip(*token_counter.most_common(max_vocab_size))
    id2token = list(vocab)
    token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
    id2token = ['<pad>', '<unk>'] + id2token
    token2id['<pad>'] = PAD_IDX 
    token2id['<unk>'] = UNK_IDX
    return token2id, id2token

token2id_combined_sent, id2token_combined_sent = build_vocab(train_all_combined_sentence_tokens)


In [12]:
# Lets check the dictionary by loading random token from it
import random 

random_token_id = random.randint(0, len(id2token_combined_sent)-1)
random_token = id2token_combined_sent[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token_combined_sent[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id_combined_sent[random_token]))

Token id 8963 ; token vestments
Token vestments; token id 8963


In [13]:
# convert token to id in the dataset.  After running this cell we will have converted the word tokens to indices
def token2index_dataset(tokens_data,token2id):
    indices_data = []
    for tokens in tokens_data:
        index_list = [token2id[token] if token in token2id else UNK_IDX for token in tokens]
        indices_data.append(index_list)
    return indices_data

train_sentence1_data_indices = token2index_dataset(train_sentence1_tokens,token2id_combined_sent)
train_sentence2_data_indices = token2index_dataset(train_sentence2_tokens,token2id_combined_sent)
val_sentence1_data_indices = token2index_dataset(val_sentence1_tokens,token2id_combined_sent)
val_sentence2_data_indices = token2index_dataset(val_sentence2_tokens,token2id_combined_sent)

# double checking
print ("Train sentence1 dataset size is {}".format(len(train_sentence1_data_indices)))
print ("Train sentence2 dataset size is {}".format(len(train_sentence2_data_indices)))
print ("Val sentence1 dataset size is {}".format(len(val_sentence1_data_indices)))
print ("Val sentence2 dataset size is {}".format(len(val_sentence2_data_indices)))

Train sentence1 dataset size is 100000
Train sentence2 dataset size is 100000
Val sentence1 dataset size is 1000
Val sentence2 dataset size is 1000


In [14]:
#visualize a random sentence1 and sentence2 paired training example
rand_training_example = random.randint(0, len(train_sentence1) - 1)
print (train_sentence1_tokens[rand_training_example])
print(train_sentence1_data_indices[rand_training_example])

print (train_sentence2_tokens[rand_training_example])
print(train_sentence2_data_indices[rand_training_example])

['a', 'boy', 'is', 'doing', 'a', 'skateboard', 'trick', 'on', 'the', 'railing', 'of', 'the', 'steps']
[2, 26, 5, 142, 2, 271, 360, 7, 3, 678, 10, 3, 402]
['a', 'boy', 'does', 'a', 'skateboard', 'trick']
[2, 26, 342, 2, 271, 360]


In [15]:
#Check average, max, min sentence lengths to determine word padding
total_sent1_len=0
total_sent2_len=0
sent1_lens=[]
sent2_lens=[]
for i in range(0,len(train_sentence1_tokens)):
    total_sent1_len+=len(train_sentence1_tokens[i])
    total_sent2_len+=len(train_sentence2_tokens[i])
    sent1_lens.append(len(train_sentence1_tokens[i]))
    sent2_lens.append(len(train_sentence2_tokens[i]))

avg1=total_sent1_len/len(train_sentence1)
avg2=total_sent2_len/len(train_sentence2)
print("sentence1 average is: "+str(avg1)+", std dev is: "+str(np.std(sent1_lens))+", max is: "+str(max(sent1_lens))+", min is: "+str(min(sent1_lens)))
print("sentence2 average is: "+str(avg2)+", std dev is: "+str(np.std(sent2_lens))+", max is: "+str(max(sent2_lens))+", min is: "+str(min(sent2_lens)))


sentence1 average is: 12.94135, std dev is: 5.755700667816214, max is: 78, min is: 2
sentence2 average is: 7.43372, std dev is: 3.0907033118046123, max is: 38, min is: 1


In [16]:
import numpy as np
import torch
from torch.utils.data import Dataset

class NewsGroupDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """
    
    def __init__(self, data_list, target_list, MAX_SENTENCE_LENGTH):
        """
        @param data_list: list of newsgroup tokens 
        @param target_list: list of newsgroup targets 

        """
        self.data_list = data_list
        self.MAX_SENTENCE_LENGTH=MAX_SENTENCE_LENGTH
        self.target_list = target_list
        assert (len(self.data_list) == len(self.target_list))

    def __len__(self):
        return len(self.data_list)
        
    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        token_idx = self.data_list[key][:self.MAX_SENTENCE_LENGTH]
        label = self.target_list[key]
        return [token_idx, len(token_idx), label]
    

In [17]:
train_dataset_sent1 = NewsGroupDataset(train_sentence1_data_indices,train_targets, 25)
train_dataset_sent2 = NewsGroupDataset(train_sentence2_data_indices,train_targets, 25)

In [18]:
print("sample sent1 lengths:")
for i in range(10):
    print(train_dataset_sent1[i][1])
print('\nsample sent2 lengths:')
for i in range(10):
    print(train_dataset_sent2[i][1])

sample sent1 lengths:
16
21
9
14
9
12
9
6
16
25

sample sent2 lengths:
9
7
9
9
9
5
8
9
4
5


In [31]:
## example output

print("sent1: x {};\ny {}".format(train_dataset_sent1[0][0], train_dataset_sent1[0][2]))
print("sent2: x {};\ny {}".format(train_dataset_sent2[0][0], train_dataset_sent2[0][2]))

sent1: x [2, 21, 23, 4, 2, 101, 20, 30, 7, 2, 720, 1934, 2, 351, 10, 50];
y 2
sent2: x [2, 21, 23, 114, 3, 807, 78, 3, 50];
y 2


In [19]:
MAX_SENTENCE_LENGTH=25

def newsgroup_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all 
    data have the same length
    """
    data_list = []
    label_list = []
    length_list = []
    #print("collate batch: ", batch[0][0])
    #batch[0][0] = batch[0][0][:MAX_SENTENCE_LENGTH]
    for datum in batch:
        label_list.append(datum[2])
        length_list.append(datum[1])
    # padding
    for datum in batch:
        padded_vec = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_SENTENCE_LENGTH-datum[1])), 
                                mode="constant", constant_values=0)
        data_list.append(padded_vec)
    return [torch.from_numpy(np.array(data_list)), torch.LongTensor(length_list), torch.LongTensor(label_list)]

In [20]:
BATCH_SIZE = 32

train_sent1_loader = torch.utils.data.DataLoader(dataset=train_dataset_sent1, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

train_sent2_loader = torch.utils.data.DataLoader(dataset=train_dataset_sent2, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_dataset_sent1 = NewsGroupDataset(val_sentence1_data_indices, val_targets, 25)
val_dataset_sent2 = NewsGroupDataset(val_sentence2_data_indices, val_targets, 25)

val_sent1_loader = torch.utils.data.DataLoader(dataset=val_dataset_sent1, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

val_sent2_loader = torch.utils.data.DataLoader(dataset=val_dataset_sent2, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=newsgroup_collate_func,
                                           shuffle=True)

In [34]:
#sent1 example
for i, (data, lengths, labels) in enumerate(train_sent1_loader):
    print(data)
    print(data.shape)
    print(lengths)
    print(lengths.shape)
    print(labels)
    print(labels.shape)
    break

tensor([[   2,   45,   11,    2,  101, 1603,    8,   31,   20,  166,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   2,    6,    4,    2,  874,  331,  789,   86,   88,   11,  329,    7,
            2, 3021,   39,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   2,   26,  161,   52,    2, 1720, 1189,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   3,    6,    4,    3,   31,   20,   17,    2, 8419, 2137,    3, 1382,
           15,    3, 2108,   11,    3,    1, 2870,    0,    0,    0,    0,    0,
            0],
        [   2,    6,  419,   38,    3,  236,   10,    2,   72,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0],
        [   2,  123,  195,   16,    2,  853,    0,    0,    0,    0,    0,    0,
            0,    0,    0,   

### Bag-of-Words model in PyTorch

Next, we will implement a Bag of Words in PyTorch -- as an `nn.Module`.

A `nn.Module` can really be any function, but it is often used to implement layers, functions and models. Note that you can also nest modules.

Importantly, modules need to have their `forward()` method overridden, and very often you will want to override the `__init__` method as well. 

The `__init__` method sets up the module. This is also often where the internal modules and parameters are initialized.

The `forward` method defines what happens when you *apply* the module.

In the background, PyTorch makes use of your code in the forward method and determines how to implement back-propagation with it - but all you need to do is to define the forward pass!

In [21]:
# First import torch related libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

class BagOfWords(nn.Module):
    """
    BagOfWords classification model
    """
    def __init__(self, vocab_size, emb_dim):
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(BagOfWords, self).__init__()
        # pay attention to padding_idx 
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.linear = nn.Linear(emb_dim,20)
    
    def forward(self, data, length):
        """
        
        @param data: matrix of size (batch_size, max_sentence_length). Each row in data represents a 
            review that is represented using n-gram index. Note that they are padded to have same length.
        @param length: an int tensor of size (batch_size), which represents the non-trivial (excludes padding)
            length of each sentences in the data.
        """
        out = self.embed(data)
        out = torch.sum(out, dim=1)
        out /= length.view(length.size()[0],1).expand_as(out).float()
     
        # return logits
        out = self.linear(out.float())
        return out

In [22]:
emb_dim = 100
model = BagOfWords(len(id2token_combined_sent), emb_dim)

In [49]:
model.linear

Linear(in_features=100, out_features=20, bias=True)

In [38]:
model.embed

Embedding(10002, 100, padding_idx=0)

In [39]:
model.embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0263,  0.7861,  0.6162,  ...,  1.2241, -1.2466,  0.7541],
        [ 0.2082, -0.5594,  1.9455,  ...,  0.9883, -1.5956, -0.8311],
        ...,
        [-0.8115, -0.8349, -0.5852,  ..., -0.8461, -0.6980, -0.5534],
        [ 0.4103,  0.8592,  0.4024,  ..., -0.4995, -0.1169, -0.4097],
        [ 1.1919, -0.2217,  0.6632,  ...,  0.4250,  0.7504,  0.7872]],
       requires_grad=True)

In [40]:
model.embed.weight.shape

torch.Size([10002, 100])

### Loss Function and Optimizer

Note that in our Bag of Words model we haven't applied softmax to the output of linear layer. Why?
We use `nn.CrossEntropyLoss()` to train. From pytorch documentation for `nn.CrossEntropyLoss()` ( https://pytorch.org/docs/stable/nn.html ) - this criterion combines `nn.LogSoftmax()` and `nn.NLLLoss()` in one single class. So, this is actually exactly the same as minimizing the log likelihood after applying softmax.

In [42]:
# Criterion and Optimizer
criterion = torch.nn.CrossEntropyLoss()  

learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [43]:
for x in model.parameters():
    print(x.shape)

torch.Size([10002, 100])
torch.Size([20, 100])
torch.Size([20])


### Training the Model

In [62]:
#this will run but not really work because we have not combined sent1, sent2 representations

num_epochs = 10 # number epoch to train

# Function for testing the model
def test_model(loader, model):
    """
    Help function that tests the model's performance on a dataset
    @param: loader - data loader for the dataset to test against
    """
    correct = 0
    total = 0
    model.eval()
    for data, lengths, labels in loader:
        data_batch, length_batch, label_batch = data, lengths, labels
        outputs = F.softmax(model(data_batch, length_batch), dim=1)
        predicted = outputs.max(1, keepdim=True)[1]
        
        total += labels.size(0)
        correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return (100 * correct / total)

for epoch in range(num_epochs):
    for i, (data, lengths, labels) in enumerate(train_sent1_loader):
        model.train()
        data_batch, length_batch, label_batch = data, lengths, labels
        optimizer.zero_grad()
        outputs = model(data_batch, length_batch)
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()
        # validate every 100 iterations
        if i > 0 and i % 100 == 0:
            # validate
            val_acc = test_model(val_sent1_loader, model)
            print('Epoch: [{}/{}], Step: [{}/{}], Validation Acc: {}'.format( 
                epoch+1, num_epochs, i+1, len(train_sent1_loader), val_acc))


Epoch: [1/10], Step: [101/3125], Validation Acc: 32.6
Epoch: [1/10], Step: [201/3125], Validation Acc: 34.8


KeyboardInterrupt: 